In [1]:
import os
from os import path

import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import pickle

from knockoff import datasets
import knockoff.config as cfg
from knockoff.adversary.policies import RandomAdversary
from knockoff.victim.blackbox import Blackbox
from knockoff.adversary.train import samples_to_transferset, get_optimizer
from knockoff.models import zoo
import knockoff.utils.model as model_utils


In [2]:
# hyper parameters
device = torch.device("cuda")

np.random.seed(cfg.DEFAULT_SEED)
torch.manual_seed(cfg.DEFAULT_SEED)
torch.cuda.manual_seed(cfg.DEFAULT_SEED)

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
queryset = datasets.ImageNet1k(
    train=True,
    transform=transform,
)

testset = datasets.ImageNet1k(train=False, transform=transform)

out_dir = "./models/adversary/ADV_DIR"
os.makedirs(out_dir, exist_ok=True)
batch_size = 32
nworkers = 10
budget = 80000

policy = "random"
victim_model = torchvision.models.resnet34(pretrained=True, progress=True).to(device)
blackbox = Blackbox.from_torchvision(victim_model)


=> done loading ImageNet1k (train) with 1024934 examples
=> done loading ImageNet1k (test) with 256233 examples


In [6]:
# get transferset
# adversary = RandomAdversary(blackbox, queryset, batch_size=batch_size)
# transferset = adversary.get_transferset(budget)
# adversary_test = RandomAdversary(blackbox, testset, batch_size=batch_size)
# transferset_test = adversary_test.get_transferset(budget=len(testset))

# pickle transfer
# with open(path.join(out_dir, 'transferset.pickle'), 'wb') as wf:
#     pickle.dump(transferset, wf)
with open(path.join(out_dir, 'transferset_test.pickle'), 'wb') as wf:
    pickle.dump(transferset_test, wf)

# or load it
# with open(path.join(out_dir, "transferset.pickle"), "rb") as rf:
#     transferset = pickle.load(rf)
# with open(path.join(out_dir, "transferset_test.pickle"), "rb") as rf:
#     transferset_test = pickle.load(rf)


In [10]:
num_classes = transferset[0][1].size(0)
budgets = [60000]

model = zoo.get_net('resnet34', 'imagenet', pretrained='imagenet',).to(device)


In [2]:
train_loader = samples_to_transferset(transferset, budget=b, transform=transform)
for i in train_loader:
    print(i)
    break

NameError: name 'samples_to_transferset' is not defined

In [13]:
test_loader = samples_to_transferset(transfer_testset, budget=len(transfer_testset), transform=transform)
for b in budgets:
    train_loader = samples_to_transferset(transferset, budget=b, transform=transform)
    optimizer = get_optimizer(model.parameters(), 'sgdm')
    criterion_train = model_utils.soft_cross_entropy
    model_utils.train_model(model, train_loader, out_dir, testset=test_loader, criterion_train=criterion_train,
                                checkpoint_suffix=f'.{b}', device=device, optimizer=optimizer)


epoch 1


/home/proj/anaconda3/envs/knockoff/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Train] Epoch: 0.11 [6336/60000 (11%)]	Loss: 0.700518	Accuracy: 93.2 (5963/6400)
[Train] Epoch: 0.21 [12736/60000 (21%)]	Loss: 0.910381	Accuracy: 92.8 (11883/12800)
[Train] Epoch: 0.32 [19136/60000 (32%)]	Loss: 0.869039	Accuracy: 92.8 (17820/19200)
[Train] Epoch: 0.43 [25536/60000 (43%)]	Loss: 0.887954	Accuracy: 92.7 (23740/25600)
[Train] Epoch: 0.53 [31936/60000 (53%)]	Loss: 0.712759	Accuracy: 92.6 (29644/32000)
[Train] Epoch: 0.64 [38336/60000 (64%)]	Loss: 0.632128	Accuracy: 92.8 (35631/38400)
[Train] Epoch: 0.75 [44736/60000 (75%)]	Loss: 0.758621	Accuracy: 92.7 (41532/44800)
[Train] Epoch: 0.85 [51136/60000 (85%)]	Loss: 0.505132	Accuracy: 92.6 (47434/51200)
[Train] Epoch: 0.96 [57536/60000 (96%)]	Loss: 0.893209	Accuracy: 92.7 (53381/57600)


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward